In [1]:
import pandas as pd
import numpy as np
from pmdarima import auto_arima
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_error, mean_squared_error
import os


# Using energy Source Datasets

# Source of Energy Natural Gas

In [2]:
# Read the Excel file
# Data_Status: Indicates the status of the data. The value "2020F" suggests that it is a forecast for the year 2020.
# State: Represents the state for which the data is recorded (in this case, "CA" for California).
# MSN: Stands for "Monthly State Names" and refers to the specific energy metric or variable being measured. Examples include ARICD, ARICV, ARTCD, ARTCV, ARTXD, WWTXV, WXICD, WXICV, ZWCDP, ZWHDP.
df = pd.read_excel('Datasets/use_energy_source.xlsx' , sheet_name='Natural Gas')


In [6]:
df.head()

,State,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AK,2034,2300,4048,5567,6805,7670,12282,11889,17636,...,339819,347228,332631,329274,333936,330883,343899,346255,343384,373567
1,AL,190706,191533,201250,211348,230316,236949,240755,262055,296690,...,609288,677380,625869,650570,701582,715033,681208,771194,748731,712874
2,AR,222202,215311,235477,252846,278852,277661,289632,295756,321569,...,288906,300572,288045,273009,296826,315617,317530,366939,371388,335437
3,AZ,140275,159093,163461,169533,174623,166063,174448,169085,189965,...,293134,339043,340375,315916,365313,373916,334629,400531,484249,513537
4,CA,1301826,1400174,1483423,1623370,1817241,1813213,1982305,2048734,2187260,...,2196252,2456371,2480792,2409574,2384061,2248939,2190994,2209801,2218732,2143958


# Data statistics

In [7]:
df.head()

,State,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AK,2034,2300,4048,5567,6805,7670,12282,11889,17636,...,339819,347228,332631,329274,333936,330883,343899,346255,343384,373567
1,AL,190706,191533,201250,211348,230316,236949,240755,262055,296690,...,609288,677380,625869,650570,701582,715033,681208,771194,748731,712874
2,AR,222202,215311,235477,252846,278852,277661,289632,295756,321569,...,288906,300572,288045,273009,296826,315617,317530,366939,371388,335437
3,AZ,140275,159093,163461,169533,174623,166063,174448,169085,189965,...,293134,339043,340375,315916,365313,373916,334629,400531,484249,513537
4,CA,1301826,1400174,1483423,1623370,1817241,1813213,1982305,2048734,2187260,...,2196252,2456371,2480792,2409574,2384061,2248939,2190994,2209801,2218732,2143958


In [8]:
df.describe()

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
count,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,...,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01
mean,4.763602e+05,4.971689e+05,5.281093e+05,5.545811e+05,5.883708e+05,6.069022e+05,6.540685e+05,6.909611e+05,7.386373e+05,7.945371e+05,...,9.597604e+05,1.002911e+06,1.030337e+06,1.052955e+06,1.083834e+06,1.092030e+06,1.079111e+06,1.198226e+06,1.239406e+06,1.212443e+06
std,1.740756e+06,1.814250e+06,1.925089e+06,2.021051e+06,2.140440e+06,2.207766e+06,2.379287e+06,2.514807e+06,2.686385e+06,2.886087e+06,...,3.450566e+06,3.607145e+06,3.705665e+06,3.783847e+06,3.894937e+06,3.921062e+06,3.872421e+06,4.300637e+06,4.449025e+06,4.353371e+06
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.580000e+02,1.870000e+02,1.970000e+02,1.670000e+02,1.960000e+02,1.620000e+02,1.630000e+02,1.920000e+02,2.280000e+02,1.990000e+02
25%,3.131800e+04,3.411425e+04,3.773125e+04,4.394200e+04,5.123450e+04,5.153375e+04,5.935425e+04,6.138750e+04,6.063950e+04,7.325500e+04,...,1.706868e+05,1.609640e+05,1.737402e+05,1.800928e+05,1.858808e+05,1.846005e+05,1.933768e+05,2.151628e+05,2.295832e+05,2.470245e+05
50%,1.416650e+05,1.526110e+05,1.610250e+05,1.667290e+05,1.742845e+05,1.764225e+05,1.875295e+05,1.924950e+05,2.020375e+05,2.251910e+05,...,2.814470e+05,2.812195e+05,2.973465e+05,3.126195e+05,3.175350e+05,3.210220e+05,3.331210e+05,3.618060e+05,3.748020e+05,3.642680e+05
75%,2.203690e+05,2.262792e+05,2.557628e+05,2.640882e+05,2.921448e+05,2.952375e+05,3.065730e+05,3.136158e+05,3.343908e+05,3.641602e+05,...,6.157452e+05,6.586028e+05,6.459890e+05,6.654460e+05,7.116898e+05,7.295430e+05,6.965265e+05,7.782842e+05,7.878492e+05,7.561342e+05
max,1.238537e+07,1.292639e+07,1.373084e+07,1.441911e+07,1.529764e+07,1.577946e+07,1.700578e+07,1.796499e+07,1.920457e+07,2.065796e+07,...,2.495377e+07,2.607569e+07,2.678877e+07,2.737684e+07,2.817969e+07,2.839279e+07,2.805689e+07,3.115387e+07,3.222455e+07,3.152352e+07


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 62 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   State   52 non-null     object
 1   1960    52 non-null     int64 
 2   1961    52 non-null     int64 
 3   1962    52 non-null     int64 
 4   1963    52 non-null     int64 
 5   1964    52 non-null     int64 
 6   1965    52 non-null     int64 
 7   1966    52 non-null     int64 
 8   1967    52 non-null     int64 
 9   1968    52 non-null     int64 
 10  1969    52 non-null     int64 
 11  1970    52 non-null     int64 
 12  1971    52 non-null     int64 
 13  1972    52 non-null     int64 
 14  1973    52 non-null     int64 
 15  1974    52 non-null     int64 
 16  1975    52 non-null     int64 
 17  1976    52 non-null     int64 
 18  1977    52 non-null     int64 
 19  1978    52 non-null     int64 
 20  1979    52 non-null     int64 
 21  1980    52 non-null     int64 
 22  1981    52 non-null     int6

In [10]:
df.head()

,State,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AK,2034,2300,4048,5567,6805,7670,12282,11889,17636,...,339819,347228,332631,329274,333936,330883,343899,346255,343384,373567
1,AL,190706,191533,201250,211348,230316,236949,240755,262055,296690,...,609288,677380,625869,650570,701582,715033,681208,771194,748731,712874
2,AR,222202,215311,235477,252846,278852,277661,289632,295756,321569,...,288906,300572,288045,273009,296826,315617,317530,366939,371388,335437
3,AZ,140275,159093,163461,169533,174623,166063,174448,169085,189965,...,293134,339043,340375,315916,365313,373916,334629,400531,484249,513537
4,CA,1301826,1400174,1483423,1623370,1817241,1813213,1982305,2048734,2187260,...,2196252,2456371,2480792,2409574,2384061,2248939,2190994,2209801,2218732,2143958


# Transformation of Data

In [11]:
df_trans = df.melt(id_vars=['State'] , var_name='Year', value_name='Yearly Data')
df_trans['Year'] = pd.to_datetime(df_trans['Year'], format='%Y')

df_trans


,State,Year,Yearly Data
0,AK,1960-01-01,2034
1,AL,1960-01-01,190706
2,AR,1960-01-01,222202
3,AZ,1960-01-01,140275
4,CA,1960-01-01,1301826
...,...,...,...
3167,WA,2020-01-01,354969
3168,WI,2020-01-01,571817
3169,WV,2020-01-01,265123
3170,WY,2020-01-01,169076


# Modeling the Data
## Using ARIMA Model

In [12]:
os.makedirs('Plots/use_energy_source_natural_gass/Arima_results_plots',exist_ok=True)

for State in df_trans['State'].unique():
        try:
            
            fig = go.Figure()

            # Get the energy consumption data for the current country and sector
            df_filter = df_trans[df_trans['State'] == State ][['Year', 'Yearly Data']]
            df_filter_index = df_filter.set_index('Year')

            train_data = df_filter[:-5]
            test_data = df_filter[-5:]
            
            # Prepare the data for modeling
            years = df_filter_index.index
            energy_consumption = df_filter_index.values.flatten()

                    # Split the data into training and testing
            # Use all data except the last 5 years for training
            Horizan = -5
            train_data = energy_consumption[:Horizan]
            test_data = energy_consumption[Horizan:]  # Use the last 5 years for testing

            # Fit the auto ARIMA model
            model = auto_arima(train_data, seasonal=False)
            model.fit(train_data)

            # Generate predictions
            predictions = model.predict(n_periods=len(test_data))
            predictions_ahead_in_future = model.predict(n_periods=len(test_data)+15)

            # Calculate evaluation metrics
            mae = mean_absolute_error(test_data, predictions)
            mse = mean_squared_error(test_data, predictions)
            mape = np.mean(np.abs((test_data - predictions) / test_data)) * 100

            print('Mean Absolute Error (MAE):', np.round(mae,2))
            print('Mean Squared Error (MSE):', np.round(mse,2))
            print('Mean Absolute Percentage Error (MAPE):', np.round(mape,2))
            
            # Plot the training data
            fig.add_trace(go.Scatter(
                x=years[:Horizan], y=train_data, mode='lines+markers', name='Training Data'))

            # Plot the predictions
            fig.add_trace(go.Scatter(
                x=years[Horizan:], y=test_data, mode='lines+markers', name='Actual'))
            fig.add_trace(go.Scatter(
                x=years[Horizan:], y=predictions, mode='lines+markers', name='Predicted'))

            fig.add_trace(go.Scatter(
                x=pd.date_range(start = years[Horizan],periods=15,freq='Y'), y=predictions_ahead_in_future, mode='lines+markers', name='Prediction till 2030'))

            # Update the layout
            fig.update_layout(title=f'Coal : Energy Consumption Forecast State : {State} using ARIMA Model',
                            xaxis_title='Year', yaxis_title='Energy Consumption')

            # Show the plot
            fig.show()
            # print(State,msn)
            fig.write_image(f'Plots/use_energy_source_natual_gass/Arima_results_plots/{State}.png')
            # break
        except:
            print('Error occoured in Combination State : {} and MSN : {} Due NaN Value'.format(State,mse))
        

Mean Absolute Error (MAE): 8225.38
Mean Squared Error (MSE): 85148909.51
Mean Absolute Percentage Error (MAPE): 2.37
Mean Absolute Error (MAE): 64528.85
Mean Squared Error (MSE): 6138125041.62
Mean Absolute Percentage Error (MAPE): 9.02
Mean Absolute Error (MAE): 54858.33
Mean Squared Error (MSE): 3848239086.01
Mean Absolute Percentage Error (MAPE): 15.57
Mean Absolute Error (MAE): 68333.0
Mean Squared Error (MSE): 7674390652.2
Mean Absolute Percentage Error (MAPE): 14.74
Mean Absolute Error (MAE): 181576.2
Mean Squared Error (MSE): 34176499564.6
Mean Absolute Percentage Error (MAPE): 8.27
Mean Absolute Error (MAE): 31658.23
Mean Squared Error (MSE): 1118635121.67
Mean Absolute Percentage Error (MAPE): 6.25
Mean Absolute Error (MAE): 15877.11
Mean Squared Error (MSE): 282599197.03
Mean Absolute Percentage Error (MAPE): 5.73
Mean Absolute Error (MAE): 4724.36
Mean Squared Error (MSE): 25644655.44
Mean Absolute Percentage Error (MAPE): 15.78
Mean Absolute Error (MAE): 14059.05
Mean Squar

## Using SARIMA Model

In [13]:
os.makedirs('Plots/use_energy_source_natural_gass/Sarima_results_plots',exist_ok=True)

for State in df_trans['State'].unique():
        try:
            
            fig = go.Figure()

            # Get the energy consumption data for the current country and sector
            df_filter = df_trans[df_trans['State'] == State ][['Year', 'Yearly Data']]
            df_filter_index = df_filter.set_index('Year')

            train_data = df_filter[:-5]
            test_data = df_filter[-5:]
            
            # Prepare the data for modeling
            years = df_filter_index.index
            energy_consumption = df_filter_index.values.flatten()

                    # Split the data into training and testing
            # Use all data except the last 5 years for training
            Horizan = -5
            train_data = energy_consumption[:Horizan]
            test_data = energy_consumption[Horizan:]  # Use the last 5 years for testing

            # Fit the auto ARIMA model
            model = auto_arima(train_data, seasonal=False)
            model.fit(train_data)

            # Generate predictions
            predictions = model.predict(n_periods=len(test_data))
            predictions_ahead_in_future = model.predict(n_periods=len(test_data)+15)

            # Calculate evaluation metrics
            mae = mean_absolute_error(test_data, predictions)
            mse = mean_squared_error(test_data, predictions)
            mape = np.mean(np.abs((test_data - predictions) / test_data)) * 100

            print('Mean Absolute Error (MAE):', np.round(mae,2))
            print('Mean Squared Error (MSE):', np.round(mse,2))
            print('Mean Absolute Percentage Error (MAPE):', np.round(mape,2))
            
            # Plot the training data
            fig.add_trace(go.Scatter(
                x=years[:Horizan], y=train_data, mode='lines+markers', name='Training Data'))

            # Plot the predictions
            fig.add_trace(go.Scatter(
                x=years[Horizan:], y=test_data, mode='lines+markers', name='Actual'))
            fig.add_trace(go.Scatter(
                x=years[Horizan:], y=predictions, mode='lines+markers', name='Predicted'))

            fig.add_trace(go.Scatter(
                x=pd.date_range(start = years[Horizan],periods=15,freq='Y'), y=predictions_ahead_in_future, mode='lines+markers', name='Prediction till 2030'))

            # Update the layout
            fig.update_layout(title=f'Coal : Energy Consumption Forecast State : {State} using SARIMA Model',
                            xaxis_title='Year', yaxis_title='Energy Consumption')

            # Show the plot
            # fig.show()
            # print(State,msn)
            fig.write_image(f'Plots/use_energy_source_natural_gass/Sarima_results_plots/{State}.png')
            # break
        except:
            print('Error occoured in Combination State : {} and MSN : {} Due NaN Value'.format(State,mse))
        # break

Mean Absolute Error (MAE): 8225.38
Mean Squared Error (MSE): 85148909.51
Mean Absolute Percentage Error (MAPE): 2.37


Mean Absolute Error (MAE): 64528.85
Mean Squared Error (MSE): 6138125041.62
Mean Absolute Percentage Error (MAPE): 9.02


Mean Absolute Error (MAE): 54858.33
Mean Squared Error (MSE): 3848239086.01
Mean Absolute Percentage Error (MAPE): 15.57


Mean Absolute Error (MAE): 68333.0
Mean Squared Error (MSE): 7674390652.2
Mean Absolute Percentage Error (MAPE): 14.74


Mean Absolute Error (MAE): 181576.2
Mean Squared Error (MSE): 34176499564.6
Mean Absolute Percentage Error (MAPE): 8.27


...

Mean Absolute Error (MAE): 12690.5
Mean Squared Error (MSE): 260336460.68
Mean Absolute Percentage Error (MAPE): 3.54


Mean Absolute Error (MAE): 49227.84
Mean Squared Error (MSE): 3264652755.2
Mean Absolute Percentage Error (MAPE): 8.68


Mean Absolute Error (MAE): 32801.8
Mean Squared Error (MSE): 1772320303.0
Mean Absolute Percentage Error (MAPE): 13.5


Mean Absolute Error (MAE): 35138.6
Mean Squared Error (MSE): 1479641799.0
Mean Absolute Percentage Error (MAPE): 20.92


Mean Absolute Error (MAE): 1537609.18
Mean Squared Error (MSE): 3396634387385.38
Mean Absolute Percentage Error (MAPE): 4.91
